In [103]:
import pickle

import cv2
import numpy as np
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import Normalizer

In [62]:
def vect_norm_x_image_data(*args):
    norm_vect_data = []
    for index, arg in enumerate(args):
        # vectorize image data by reshaping the axes
        vectorized_shape = (arg.shape[0], arg.shape[1] * arg.shape[2])
        vect_arg = np.array(arg).reshape(vectorized_shape)
        # improve performance by normalizing data
        norm_vect_data.append(Normalizer().transform(vect_arg))
    return norm_vect_data

def unpickle_model(file_name):
    with open(file_name, 'rb') as file:
        return pickle.load(file)

In [111]:
_, (x_test, y_test) = tf.keras.datasets.mnist.load_data(path="mnist.npz")
x_test, = vect_norm_x_image_data(x_test)
print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}\n")

x_test shape: (10000, 784)
y_test shape: (10000,)



In [ ]:
k_neighbors: KNeighborsClassifier = unpickle_model('kneighbors_digit_model.dat')
y_pred = k_neighbors.predict(x_test)
score = k_neighbors.score(x_test, y_test)
print(f'Score = {score}')
print('Predicted Actual')
print('--------- ------')
for i, prediction in enumerate(y_pred):
    if i == 100:
        break
    print('{0:9} {1:6}'.format(prediction, y_test[i]))

In [ ]:
random_forest: RandomForestClassifier = unpickle_model('randomforest_digit_model.dat')
y_pred = random_forest.predict(x_test)
score = random_forest.score(x_test, y_test)
print(f'Score = {score}')
print('Predicted Actual')
print('--------- ------')
for i, prediction in enumerate(y_pred):
    if i == 100:
        break
    print('{0:9} {1:6}'.format(prediction, y_test[i]))

In [139]:
def predict(image):
    img = cv2.imread(image, 0)
    img = cv2.resize(img, (28, 28))  # resize: model trained on 28x28
    img = cv2.bitwise_not(img)  # invert black and white

    img = img.reshape(img.shape[0] * img.shape[1])
    img = img/255  # scale 255 to 1
    img = np.array([img])

    pred = random_forest.predict(img)[0]
    pred_prob = random_forest.predict_proba(img)
    print(f'Random forest prediction: {pred}')
    print(f'{pred_prob}\n')

    pred = k_neighbors.predict(img)[0]
    pred_prob = k_neighbors.predict_proba(img)
    print(f'K neighbors prediction: {pred}')
    print(f'{pred_prob}\n\n')